In [9]:
import torch
import torch.nn as nn
import numpy as np
from utilis_ForsaV2 import *
import utilis_ForsaV2
# !pip install fxpmath\
from fxpmath import Fxp
import time
word_size  = 8
frac_size = 6
utilis_ForsaV2.word_size = word_size
utilis_ForsaV2.frac_size = frac_size
# from tensorboardX import SummaryWriter
# from transforms import *
# from models_cust import *


In [10]:
PATH = './PretrainedModels/LeNetPretrainedCifar10.pth'
model = torch.load(PATH)
# model = torch.hub.load('pytorch/vision:v0.10.0', 'mobilenet_v2', pretrained=True)
model.eval()

RecursiveScriptModule(
  original_name=Net
  (conv1): RecursiveScriptModule(original_name=Conv2d)
  (pool): RecursiveScriptModule(original_name=MaxPool2d)
  (conv2): RecursiveScriptModule(original_name=Conv2d)
  (fc1): RecursiveScriptModule(original_name=Linear)
  (fc2): RecursiveScriptModule(original_name=Linear)
  (fc3): RecursiveScriptModule(original_name=Linear)
)

In [11]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
print(count_parameters(model))

159518


# Demonstrating sorting for LeNet
a) Regular convolution: 

In [12]:
No_layers=4
sw_wt = np.zeros(No_layers)
sw_bias = np.zeros(No_layers)
sw_wt_sorted = np.zeros(No_layers)
sw_bias_sorted = np.zeros(No_layers)
########## sorting features.0 ##########
_, new_indx1 = sortFullMatrix_V2(model.conv1.weight)
model.conv1.weight.data, _, sw_wt[0], sw_wt_sorted[0]  = compare_sw_sort(model.conv1.weight.data, new_indx1, -3, printsumm=True)
model.conv1.bias.data, _, sw_bias[0], sw_bias_sorted[0]  = compare_sw_sort(model.conv1.bias.data, new_indx1, -1)
# plot_dist(model.features[0][0].weight)
########## rearranging features.3 ##########
model.conv2.weight.data = model.conv2.weight[:, new_indx1, :, :]

# ########## sorting features.3 ##########
_, new_indx1 = sortFullMatrix_V2(model.conv2.weight)
model.conv2.weight.data, _, sw_wt[1], sw_wt_sorted[1]  = compare_sw_sort(model.conv2.weight.data, new_indx1, -3, printsumm=True)
model.conv2.bias.data, _, sw_bias[1], sw_bias_sorted[1]  = compare_sw_sort(model.conv2.bias.data, new_indx1, -1)


Switching before sorting 3263.0
Switching after sorting 2706.0
Percentage of switching reduction 17.070180815200736 %
Switching before sorting 35629.0
Switching after sorting 31920.0
Percentage of switching reduction 10.410059221420752 %


In [13]:
########## rearranging classifier.1 ##########
# Reshaping first classifier weight to rearrange prior to its sort
fc1_weight = model.fc1.weight
fc1_weight = fc1_weight.view(120, 32, 6, 6)
new_fc1_weight = fc1_weight[:, new_indx1, :, :]
model.fc1.weight.data = new_fc1_weight.view(new_fc1_weight.shape[0], -1) 

In [14]:
# ########## sorting classifier.1 ##########
_, new_indx1 = sortFullMatrix_V2(model.fc1.weight)
model.fc1.weight.data, _, sw_wt[2], sw_wt_sorted[2]  = compare_sw_sort(model.fc1.weight.data, new_indx1, -1, isconv=False, printsumm=True)
model.fc1.bias.data, _, sw_bias[2], sw_bias_sorted[2]  = compare_sw_sort(model.fc1.bias.data, new_indx1, -1, isconv=False)
########## rearranging classifier.4 ##########
model.fc2.weight.data = model.fc2.weight[:, new_indx1]


Switching before sorting 469794.0
Switching after sorting 416965.0
Percentage of switching reduction 11.245141487545606 %


In [15]:
# Choose either to sort along 84 in fc3 or fc2. It could be beneficial to sort fc3 along 83 and rearrange fc2 based on the indices obtained
# ########## sorting classifier.1 ##########
_, new_indx1 = sortFullMatrix_V2(model.fc2.weight)
model.fc2.weight.data, _, sw_wt[3], sw_wt_sorted[3]  = compare_sw_sort(model.fc2.weight.data, new_indx1, -1, isconv=False)
model.fc2.bias.data, _, sw_bias[3], sw_bias_sorted[3]  = compare_sw_sort(model.fc2.bias.data, new_indx1, -1, isconv=False)
########## rearranging classifier.4 ##########
model.fc3.weight.data = model.fc3.weight[:, new_indx1]


In [17]:
# If the whole flattened weight matrix is sorted along its filter dimension, no indexing is required. 
# If slices of the matrix is sorted indexing is needed to keep track the channels of the filter
# Here the slices of the flattened filter are sorted independently to demonstrate the tradeoff 
# between switching activity reduction and the indexing overhead
############### Run cell after cell [43]
# To sort fc3 before fc2, we need to transpose 10x84 to 84x10 and sort the matrix
fc3_trans = torch.transpose(model.fc2.weight.data, 0, 1)
_, new_indxfc3 = sortFullMatrix_V2(fc3_trans)
fc3_untrans, _, sw_wt[3], sw_wt_sorted[3]  = compare_sw_sort(torch.transpose(model.fc2.weight.data, 0, 1), new_indxfc3, -1, isconv=False, printsumm=True)
model.fc3.weight.data = torch.transpose(fc3_untrans, 0, 1)
###################################### How about slicing
_, new_indx1 = sortFullMatrix_V2(model.conv1.weight.data[:, :,0, 0])
slice1, _, sw_wt_1, sw_wt_sorted_1  = compare_sw_sort(model.conv1.weight.data[:, :, 0, 0], new_indx1, -1)
_, new_indx1 = sortFullMatrix_V2(model.conv1.weight.data[:, 0, :, 0])
slice2, _, sw_wt_2, sw_wt_sorted_2  = compare_sw_sort(model.conv1.weight.data[:, 0, :, 0], new_indx1, -1)
_, new_indx1 = sortFullMatrix_V2(model.conv1.weight.data[:, 0, 0, :])
slice3, _, sw_wt_3, sw_wt_sorted_3  = compare_sw_sort(model.conv1.weight.data[:, 0, 0, :], new_indx1, -1)
print(sw_wt_1, 'first', sw_wt_sorted_1)
print(sw_wt_2, 'Second', sw_wt_sorted_2)
print(sw_wt_3, 'Third', sw_wt_sorted_3)


Switching before sorting 37067.0
Switching after sorting 32766.0
Percentage of switching reduction 11.603312919847843 %
307.0 first 196.0
302.0 Second 198.0
296.0 Third 188.0
